<h2>ARIMA</h2>

In [1]:
import quandl
import warnings
import itertools
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('whitegrid')
sns.set_context('talk')

from statsmodels.tsa.seasonal import seasonal_decompose

from arima_utils import ad_fuller_test, plot_rolling_stats
from arima_utils import plot_acf_pacf, arima_gridsearch_cv

In [3]:
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}

%matplotlib inline
plt.rcParams.update(params)

# Specify to ignore warning messages
warnings.filterwarnings("ignore")

<h2>Getting the Data</h2>

In [6]:
df = quandl.get("BUNDESBANK/BBK01_WT5511", end_date="2017-07-31")
print(df.shape)

new_df = df.reindex(pd.date_range(fd.index.min(), df.index.max(), freq='D')).fillna(method='ffill')
print(new_df.shape)

ConnectionError: HTTPSConnectionPool(host='www.quandl.com', port=443): Max retries exceeded with url: /api/v3/datasets/BUNDESBANK/BBK01_WT5511/data?order=asc&end_date=2017-07-31 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f0248398730>: Failed to establish a new connection: [Errno 110] Connection timed out'))

In [ ]:
new_df.plot(figsize=(15, 6))
plt.show()

<h2>Decompose</h2>

In [ ]:
# Interpolation is a method for generating points between given points.
decompose = seasonal_decompose(new_df.Value.interpolate())
decompose.plot()

<h2>Dickey Fuller Test</h2>
</br>Original Series

In [ ]:
# Original Series
ad_fuller_test(new_df.Value)
plot_rolling_stats(new_df.Value)

<h3>Log Series</h3>

In [ ]:
log_series = np.log(new_df.Value)
ad_fuller_test(log_series)
plot_rolling_stats(log_series)

<h3>Log Difference<h3>

In [ ]:
# Using log series with a shift to make it stationary
# The shift() function is used to shift index by desired number of periods with an optional time freq. Number of periods to shift. Can be positive or negative.
log_series_shift = log_series - log_series.shift()
log_series_shift = log_series_shift[~np.isnan(log_series_shift)]

In [ ]:
ad_fuller_test(log_series_shift)
plot_rolling_stats(log_series_shift)

<h3>ACF/PACF</h3>

In [ ]:
plot_acf_pacf(log_series_shift)

<h3>ARIMA</h3>

In [7]:
new_df['log_series'] = log_series
new_df['log_series_shift'] = log_series_shift

NameError: name 'log_series' is not defined

<h3>Traning-Testing Split</h3>

In [ ]:
result_dict = arima_gridsearch_cv(new_df.log_series, cv_split=5)